<a href="https://colab.research.google.com/github/yinghuing/K_drama_recommender/blob/master/chapter4/implement_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 0: ハンズオンの準備
---

## 必要なライブラリのインストール

In [ ]:
# !pip install -q langchain langchain-openai langchain-community langchain-core langchain-text-splitters

In [6]:
!pip install -q langchain==0.3.7 langchain-openai==0.2.9 langchain-community==0.3.7 langchain-core==0.3.18 langchain-text-splitters==0.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [7]:
!pip freeze | grep langchain

langchain==0.3.7
langchain-community==0.3.7
langchain-core==0.3.18
langchain-openai==0.2.9
langchain-text-splitters==0.3.2


## API キーの設定

*  左ナビゲーションで [**シークレット**] アイコン (鍵形のアイコン) をクリックします。
*  [**新しいシークレットを追加**] をクリックし、`LANGCHAIN_API_KEY`、`OPENAI_API_KEY`、`PINECONE_API_KEY` の 3 つを設定し、[**ノートブックからのアクセス**] を有効にします。
  *  `OPENAI_API_KEY` の [**値**] には指定されたキーを入力します。
  *  `LANGCHAIN_API_KEY` と `PINECONE_API_KEY` の [**値**] にはご自身で取得したキーを入力してください。
*  入力が完了したら、下のセルを実行します。

In [8]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "default"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

## サンプルファイルのアップロード

*  左ナビゲーションで [**ファイル**] アイコンをクリックします。
*  開いた [ファイル] 欄の何もない部分で右クリックし、[**新しいフォルダ**] をクリックします。(sample_data と同じ階層にフォルダが作られるようにします)
*  作成されたフォルダに **files** という名前を設定します。
*  files フォルダにカーソルを合わせ、3 点リーダアイコンをクリックして、[**アップロード**] をクリックします。
*  ローカルの files フォルダにあるすべてのファイルを選択してアップロードします。
*  「警告」のポップアップが表示されますが問題ありません。[**OK**] をクリックしてポップアップを閉じます。

In [9]:
import warnings
warnings.filterwarnings('ignore')

# Section 1: LangChain による RAG の実装


---



## Document loaders
---
https://python.langchain.com/docs/concepts/#document-loaders
https://python.langchain.com/docs/integrations/document_loaders/

### TextLoader - テキストファイルのロード
[langchain_community.document_loaders.text.TextLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.text.TextLoader.html)

In [17]:
from langchain_community.document_loaders.text import TextLoader

file = 'files/gingatetsudono_yoru.txt'
print(f'Loading {file}')
loader = TextLoader(file)
data = loader.load() # Document オブジェクトのリストが返される

# data
print(data[0].page_content)

Loading files/gingatetsudono_yoru.txt
銀河鉄道の夜
宮沢賢治


一、午后の授業

「ではみなさんは、さういふふうに川だと云はれたり、乳の流れたあとだと云はれたりしてゐたこのぼんやりと白いものがほんたうは何かご承知ですか。」先生は、黒板に吊した大きな黒い星座の図の、上から下へ白くけぶった銀河帯のやうなところを指しながら、みんなに問をかけました。
カムパネルラが手をあげました。それから四五人手をあげました。ジョバンニも手をあげやうとして、急いでそのまゝやめました。たしかにあれがみんな星だと、いつか雑誌で読んだのでしたが、このごろはジョバンニはまるで毎日教室でもねむく、本を読むひまも読む本もないので、なんだかどんなこともよくわからないといふ気持ちがするのでした。
ところが先生は早くもそれを見附けたのでした。
「ジョバンニさん。あなたはわかってゐるのでせう。」
ジョバンニは勢よく立ちあがりましたが、立って見るともうはっきりとそれを答へることができないのでした。ザネリが前の席からふりかへって、ジョバンニを見てくすっとわらひました。ジョバンニはもうどぎまぎしてまっ赤になってしまひました。先生がまた云ひました。
「大きな望遠鏡で銀河をよっく調べると銀河は大体何でせう。」
　やっぱり星だとジョバンニは思ひましたがこんどもすぐに答へることができませんでした。
先生はしばらく困ったやうすでしたが、眼をカムパネルラの方へ向けて、「ではカムパネルラさん。」と名指しました。するとあんなに元気に手をあげたカムパネルラが、やはりもぢもぢ立ち上ったまゝやはり答へができませんでした。
先生は意外なやうにしばらくぢっとカムパネルラを見てゐましたが、急いで「では。よし。」と云ひながら、自分で星図を指しました。
「このぼんやりと白い銀河を大きないゝ望遠鏡で見ますと、もうたくさんの小さな星に見えるのです。ジョバンニさんさうでせう。」
ジョバンニはまっ赤になってうなづきました。けれどもいつかジョバンニの眼のなかには涙がいっぱいになりました。さうだ僕は知ってゐたのだ、勿論カムパネルラも知ってゐる、それはいつかカムパネルラのお父さんの博士のうちでカムパネルラといっしょに読んだ雑誌のなかにあったのだ。それどこでなくカムパネルラは、その雑誌を読むと、すぐお父さんの書〔斎〕から巨

### PyPDFLoader - PDF ファイルのロード
[langchain_community.document_loaders.pdf.PyPDFLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)



In [18]:
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.7 MB/s eta 0:00:00


In [19]:
from langchain_community.document_loaders.pdf import PyPDFLoader

file = 'files/employment_regulations.pdf'
print(f'Loading {file}')
loader = PyPDFLoader(file)
data = loader.load()

# data
print(data[0].page_content)

Loading files/employment_regulations.pdf
LangChain Training 社 就業規定 
第 1 章 総則 
第 1 条 （目的） 
本規定は、LangChain Training 社（以下「当社」という）の従業員の就業に関する基
本的な事項を定め、従業員の適正かつ円滑な業務運営を図ることを目的とします。 
第 2 条 （適用範囲） 
本規定は、当社に勤務する全ての従業員に適用されます。 
第 2 章 勤務時間・休憩時間 
第 3 条 （勤務時間） 
1. 通常の勤務時間は、午前 9 時から午後 6 時までとします。 
2. 所定の勤務時間は 1 日 8 時間、週 40 時間とします。 
第 4 条 （休憩時間） 
1. 休憩時間は、勤務時間の途中に 1 時間とします。 
2. 休憩時間は原則として、午後 12 時から午後 1 時までとします。 
第 5 条 （残業） 
1. 業務上の必要により、勤務時間を超えて残業を命じる場合があります。 
2. 残業を行う場合は、事前に上司の許可を得なければなりません。 
第 3 章 休暇・休業 
第 6 条 （年次有給休暇） 
1. 従業員は、入社後 6 か月を経過した時点で、10 日の有給休暇を取得する権利を
有します。 
2. 有給休暇は、年度ごとに付与され、繰り越しはできません。 
第 7 条 （特別休暇） 
従業員は以下の理由により、特別休暇を申請することができます。 


### Docx2txtLoader - DOCX ファイルのロード
[langchain_community.document_loaders.word_document.Docx2txtLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.word_document.Docx2txtLoader.html)

In [20]:
!pip install -q docx2txt

  Preparing metadata (setup.py) ... done


In [21]:
from langchain_community.document_loaders.word_document import Docx2txtLoader

file = 'files/the_great_gatsby.docx'
print(f'Loading {file}')
loader = Docx2txtLoader(file)
data = loader.load()

# print(data[0].page_content)

Loading files/the_great_gatsby.docx


### WikipediaLoader
[langchain_community.document_loaders.wikipedia.WikipediaLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html)

In [22]:
!pip install -q wikipedia

  Preparing metadata (setup.py) ... done


In [23]:
from langchain_community.document_loaders.wikipedia import WikipediaLoader

query = 'ディープラーニング'
lang = 'ja'
load_max_docs = 2

loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
data = loader.load()

print(data[0].page_content)

ディープラーニング（英: deep learning）または深層学習（しんそうがくしゅう）とは、対象の全体像から細部までの各々の粒度の概念を階層構造として関連させて学習する手法のことである。深層学習は複数の独立した機械学習手法の総称であり、その中でも最も普及した手法は、（狭義には4層以上の）多層の人工ニューラルネットワーク（ディープニューラルネットワーク、英: deep neural network; DNN）による機械学習手法である。
要素技術としてはバックプロパゲーションなど、20世紀のうちに開発されていたものの、4層以上の深層ニューラルネットについて、局所最適解や勾配消失などの技術的な問題によって十分学習させられず、性能も芳しくなかった。しかし、ジェフリー・ヒントンの研究チームが2006年に多層ニューラルネットワークを用いたオートエンコーダを発表したことをきっかけに、多層ニューラルネットワークの学習の研究が進展し、同時に学習に必要な計算機の能力向上とインターネットの発展による学習データの流通が相まって、十分に学習させられるようになった。その結果、音声・画像・自然言語を対象とする諸問題に対して他の手法を圧倒する高い性能を示し、2000年代末から2010年代にかけて急速に普及した。
深層学習が機械学習分野に与えた影響は非常に大きく、2015年に発表された拡散モデルに代表される生成モデルの多くに組み込まれたほか、2017年に発表されたTransformerをはじめとする大規模言語モデルなどの基盤にもなっている。しかし、深層学習によって行われたパターン認識がどのような根拠に基づいているかを解析することは難しく、ブラックボックス問題を引き起こしている。また、開発競争がきわめて激しく、最新の手法が数ヶ月で古くなるような事もあり得る状況であるため、常に最先端の技術を追いかけ続けることは容易ではない。


== 概要 ==
ディープラーニングは、学習に用いる具体的な数学的概念はどうであれ、対象の全体像から細部までの各々の粒度の概念を階層構造として関連させて学習する手法を指す。21世紀に入って、オートエンコーダを始めとするジェフリー・ヒントンらによる多層ニューラルネットワークによる学習の研究や、学習に必要な計算機の能力向上、および、インターネットの発展による学習データの流通によ

これら以外にも、様々なソースからデータを取得するための Document loader を利用できる。  
https://python.langchain.com/docs/integrations/document_loaders/

あらためて、サンプルの社内規程を PDF からロードする。

In [24]:
from langchain_community.document_loaders.pdf import PyPDFLoader

file = 'files/employment_regulations.pdf'
print(f'Loading {file}')
loader = PyPDFLoader(file)
data = loader.load()

Loading files/employment_regulations.pdf


## Text splitters
---
https://python.langchain.com/docs/concepts/#text-splitters  
https://python.langchain.com/docs/how_to/#text-splitters
  
---
### チャンク分割のテスト用サイト
- [ChunkViz](https://chunkviz.up.railway.app/)  
- [Text Splitter Playground](https://langchain-text-splitter.streamlit.app/)

### RecursiveCharacterTextSplitter
[langchain_text_splitters.character.RecursiveCharacterTextSplitter](https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=0
)

chunks = text_splitter.split_documents(data) # 各チャンクの Document のリストを返す

# chunks
# print(len(chunks))
print(chunks[1].page_content)

1. 休憩時間は、勤務時間の途中に 1 時間とします。 
2. 休憩時間は原則として、午後 12 時から午後 1 時までとします。 
第 5 条 （残業） 
1. 業務上の必要により、勤務時間を超えて残業を命じる場合があります。 
2. 残業を行う場合は、事前に上司の許可を得なければなりません。 
第 3 章 休暇・休業 
第 6 条 （年次有給休暇） 
1. 従業員は、入社後 6 か月を経過した時点で、10 日の有給休暇を取得する権利を
有します。 
2. 有給休暇は、年度ごとに付与され、繰り越しはできません。 
第 7 条 （特別休暇）


## Embedding models
---  
https://python.langchain.com/docs/concepts/#embedding-models  
https://python.langchain.com/docs/integrations/text_embedding/


### tiktoken を使って Embedding のコストを見積もる
tiktoken は Text splitter で分割したチャンクのリストを入力として受け取り、指定のモデルで embedding を実行した場合に処理されるトークン数を算出して出力する。

In [26]:
import tiktoken
enc = tiktoken.encoding_for_model('text-embedding-3-small')
total_tokens = sum([len(enc.encode(page.page_content)) for page in chunks])
# check prices here: https://openai.com/pricing
print(f'Total Tokens: {total_tokens}')
print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')

Total Tokens: 12236
Embedding Cost in USD: 0.000245


### OpenAIEmbeddings
[langchain_openai.embeddings.base.OpenAIEmbeddings](https://python.langchain.com/api_reference/openai/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html)

In [27]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)

*   Embidding model が作成される。
*   ここではまだ embedding の処理が実行されたわけではなく、次のステップで Vector store に Index を作成する際にこの Embedding model を使用する。

### 【参考】embedding によってベクトル化されたデータはどのような形をしているか

In [28]:
text = "ディープラーニング（英: deep learning）または深層学習（しんそうがくしゅう）とは、対象の全体像から細部までの各々の粒度の概念を階層構造として関連させて学習する手法のことである。"

# embed_query メソッドはテキストデータを入力として受け取り、変換されたベクトルを出力する
vector = embeddings.embed_query(text)

print(len(vector))
print(vector)

1536
[-0.031940292567014694, -0.04326494410634041, 0.013767024502158165, -0.014913445338606834, 0.04366369917988777, 0.012381349690258503, 0.015820613130927086, 0.059893034398555756, -0.040493592619895935, 0.03255836293101311, 0.0012224339880049229, -0.018312834203243256, -0.008149559609591961, -0.0365060418844223, 0.040393903851509094, -0.013816868886351585, -0.003399388398975134, -0.0068884966894984245, 0.05131979659199715, 0.0006623074878007174, 0.0247028861194849, -0.06990179419517517, 0.04386307671666145, -0.011185084469616413, 0.07117781043052673, -0.0498444028198719, 0.07436785101890564, 0.016897253692150116, -0.0007906568353064358, 0.01896081119775772, 0.028491061180830002, -0.013627460226416588, -0.04773100093007088, -0.008249248377978802, 0.03417332470417023, -0.04517896845936775, -0.015700986608862877, 0.061408303678035736, -0.05929490178823471, 0.051878053694963455, 0.015581361018121243, -0.025340894237160683, -0.007795664947479963, 0.04410232976078987, 0.020067356526851654

OpenAI 以外にも、クラウド等で提供される様々な Embeddings model を利用することができる。  
https://python.langchain.com/docs/integrations/text_embedding/

## Vector stores (Pinecone を使用する場合)
---
https://python.langchain.com/docs/concepts/#vector-stores  
https://python.langchain.com/docs/integrations/vectorstores/  
https://python.langchain.com/docs/integrations/vectorstores/pinecone/  
[langchain_pinecone.vectorstores.PineconeVectorStore](https://python.langchain.com/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html)

In [ ]:
# !pip install -q pinecone langchain-pinecone

In [ ]:
# !pip freeze | grep pinecone

In [29]:
!pip install -q pinecone==5.3.1 langchain-pinecone==0.2.0

In [30]:
import pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from pinecone import PodSpec, ServerlessSpec

### 既存の Index を削除する
Pinecone の無料プランでは Index を 1 つしか作成できないため、既存の Index がある場合はいったん削除する。

In [31]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()

    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')

In [32]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


### Index と Vector store の作成

In [33]:
pc = pinecone.Pinecone()

index_name = 'regulation'

 # creating the index and embedding the chunks into the index
print(f'Creating index {index_name} and embeddings ... ')

# creating a new index
pc.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

# processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
# inserting the embeddings into the index and returning a new Pinecone vector store object.
vector_store = PineconeVectorStore.from_documents(chunks, embeddings, index_name=index_name)

print('Ok')

Creating index regulation and embeddings ... 
Ok


Pinecone 以外にも様々な Vector store を利用することができる。  
https://python.langchain.com/docs/integrations/vectorstores/  

これ以外にも、各クラウドで Vector store として利用できるデータベースが提供されており、LangChain で使用するためのライブラリも提供されている。


## Retrievers
---
https://python.langchain.com/docs/concepts/#retrievers

### Retriever の作成

In [34]:
# 類似度の高いチャンクを何個取得するかを指定
k = 5

# Vector store から Retriever を作成
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

## Chain
---

### Chat model と Prompt template の作成

In [35]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [36]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [37]:
chat_prompt = ChatPromptTemplate.from_template("""以下の参考用のテキストの一部を参照して、質問に回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。

{context}

質問：{query}""")

In [38]:
system_message = """以下の参考用のテキストの一部を参照して、質問に回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。

{context}"""
human_message = "質問：{query}"

chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system", system_message
    ),
    (
        "human", human_message
    )
])

### Chain の作成と実行  
上で作成したコンポーネントを組み合わせて Chain を定義し、実行する

In [48]:
# Chain を定義
qa_chain = (
    {
        "query":RunnablePassthrough(),
        "context":retriever
    }
    |chat_prompt
    |model
    |StrOutputParser()
)

# クエリ
query = "国内出張の宿泊費の上限はいくらですか。"
# query = "育児休暇を取ることはできますか。"
# query = "海外出張の出張手当は1日当たりいくらですか。"
# query = "介護休職中に会社から何らかのサポートを受けることはできますか。"


# Chain の実行
result = qa_chain.invoke(query)
# result = qa_chain.invoke(query, config={'callbacks': [ConsoleCallbackHandler()]})
print(result)

国内出張の宿泊費の上限は、地域および宿泊施設のランクに応じて以下の通り定められています。

- 都市部（東京、大阪、名古屋など）：15,000円/泊
- 地方都市：12,000円/泊
- その他地域：10,000円/泊


#### Chain への入力を辞書にする場合

In [49]:
from operator import itemgetter
qa_chain = (
    {
        "query": itemgetter("query")|RunnablePassthrough(),
        "context": itemgetter("query")|retriever
    }
    |chat_prompt
    |model
    |StrOutputParser()
)

query = "国内出張の宿泊費の上限はいくらですか。"

# result = qa_chain.invoke(query, config={'callbacks': [ConsoleCallbackHandler()]})
result = qa_chain.invoke({"query": query})
print(result)

国内出張の宿泊費の上限は、地域および宿泊施設のランクに応じて以下の通り定められています。

- 都市部（東京、大阪、名古屋など）：15,000円/泊
- 地方都市：12,000円/泊
- その他地域：10,000円/泊


### RunnableParallel

以下の部分で何をやっているか。
```
qa_chain = (
    {
        "query":RunnablePassthrough(),
        "context":retriever
```
この部分は、以下の書き方の省略形になっている。
```
qa_chain = (
    RunnableParallel({
      "query":RunnablePassthrough(),
      "context": retriever})
    )
```
`RunnableParallel` は複数の Runnable に同じ入力を渡して並行して実行し、結果をそれぞれのキーの値として格納した辞書を出力する。  
[langchain_core.runnables.base.RunnableParallel](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableParallel.html)  

```
result = qa_chain.invoke(query)
```
ここで `qa_chain` に入力された `query` は、`RunnableParallel` に渡され、`retriever` によって `query` と類似したドキュメントが検索されて、プロンプトの変数 `context` に代入される値として `chat_prompt` に渡される。並行して、`query` の値はプロンプトの変数 `query` に代入されるべき値として `chat_prompt` に渡される。  

`RunnablePassthrough` は受け取った入力をそのまま出力する。  
`qa_chain` の 2 番目にある `chat_prompt` はユーザーの入力を直接受け取ることができないため、`RunnablePassthrough` によってユーザーの入力を受け渡される必要がある。  
[langchain_core.runnables.passthrough.RunnablePassthrough](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html)



#### RunnableParallel の動作
*  `RunnableParallel` も Runnable であり、`invoke` メソッドで実行できる。
*  ここでは `1` という入力に対して 2 つの処理を実行する例で動作を確認してみる。

In [41]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1)

{'origin': 1, 'modified': 2}

# Section 2: RAG の処理を関数化して実行する
---

## 関数を定義

### ドキュメントのロード

In [42]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain_community.document_loaders.pdf import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain_community.document_loaders.word_document import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain_community.document_loaders.text import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data

In [43]:
# wikipedia
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

### RecursiveCharacterTextSplitter での処理を関数にラップする

In [44]:
def chunk_data(data, chunk_size=256, chunk_overlap=0):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(data)
    return chunks

### Embedding のコスト計算

In [45]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')

### Vector store
新規の Index を作成、あるいは既存の Index をロードして、Vector store オブジェクトを作成する。

In [46]:
def insert_or_fetch_embeddings(index_name, chunks):
    # Importing the necessary libraries and initializing the Pinecone client
    import pinecone
    from langchain_pinecone import PineconeVectorStore
    from langchain_openai import OpenAIEmbeddings
    from pinecone import PodSpec, ServerlessSpec


    pc = pinecone.Pinecone()

    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)

    # Loading from existing index
    if index_name in pc.list_indexes().names():
        print(f'Index {index_name} already exists. Loading embeddings ... ')
        vector_store = PineconeVectorStore.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        # Creating the index and embedding the chunks into the index
        print(f'Creating index {index_name} and embeddings ... ')

        # Creating a new index
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

        # Processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
        # Inserting the embeddings into the index and returning a new Pinecone vector store instance.
        vector_store = PineconeVectorStore.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')

    return vector_store

### Index の削除 (上と同じ)

In [47]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()

    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')

### RAG Chain

In [50]:
def get_answer(vector_store, q, k=5):
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.runnables import RunnablePassthrough, RunnableParallel
    from langchain_openai import ChatOpenAI
    from operator import itemgetter
    from langchain_core.output_parsers import StrOutputParser

    # Chat model
    model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    # Retriever
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    # Prompt template
    system_message = """以下の参考用のテキストの一部を参照して、質問に回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。

    {context}"""
    human_message = "質問：{query}"

    chat_prompt = ChatPromptTemplate.from_messages([

        (
            "system", system_message
        ),
        (
            "human", human_message
        )
    ])

    # Chain の定義
    chain = (
        {
            "query":itemgetter('query')|RunnablePassthrough(),
            "context":itemgetter('query')|retriever
        }
        |chat_prompt
        |model
        |StrOutputParser()
    )

    # Chain の実行
    answer = chain.invoke({'query':q})
    return answer

## 処理の実行  
上で定義した関数を実行する

### load_document/load_wikipedia を実行

In [51]:
data = load_document('files/employment_regulations.pdf')

# print(data[1].page_content)
# print(data[10].metadata)

print(f'You have {len(data)} pages in your data')
print(f'There are {len(data[10].page_content)} characters in the page')

Loading files/employment_regulations.pdf
You have 20 pages in your data
There are 630 characters in the page


### chunk_data を実行

In [52]:
chunks = chunk_data(data, chunk_size=300, chunk_overlap=0)
print(len(chunks))

50


In [53]:
print_embedding_cost(chunks)

Total Tokens: 12236
Embedding Cost in USD: 0.000245


### delete_pinecone_index を実行

In [54]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


### insert_or_fetch_embeddings を実行

In [55]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)

Creating index askadocument and embeddings ... 
Ok


### get_answer をインタラクティブな形で実行

In [56]:
import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break

    answer = get_answer(vector_store, q)
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')

Write Quit or Exit to quit.
Question #1: 出張費用

Answer: 出張費用は以下の項目で構成されています。

1. **交通費**: 出張に必要な公共交通機関の実費を支給します。最も経済的かつ合理的な経路および方法を利用することが原則です。

2. **宿泊費**: 出張先で宿泊が必要な場合に支給され、地域および宿泊施設のランクに応じて上限額が定められています。
   - 都市部（東京、大阪、名古屋など）：15,000円/泊
   - 地方都市：12,000円/泊
   - その他地域：10,000円/泊
   宿泊費の支給には領収書の提出が必要です。

3. **日当**: 出張に伴う通常の食事代や雑費に対して支給されます。日帰り出張の場合は2,000円/日、宿泊を伴う出張の場合は3,000円/日が支給されます。

4. **その他必要経費**: 出張に際して発生する通信費、資料代、ビザ申請費用などが含まれ、会社が認めた場合に支給されます。これらの経費についても領収書の提出が必要です。

旅費の支給を受けるためには、出張終了後速やかに所定の旅費精算書を提出し、交通費、宿泊費、その他の経費に関する領収書を添付する必要があります。

 -------------------------------------------------- 

Question #2: 海外出張　アメリカ

Answer: 分からない。

 -------------------------------------------------- 

Question #3: 海外出張費用の上限　アメリカ

Answer: アメリカの宿泊費の上限は、北米、欧州主要都市の場合、USD 300/泊と定められています。また、日当はUSD 100/日です。

 -------------------------------------------------- 

Question #4: quit
Quitting ... bye bye!


# Section 3: RAG Chain に会話履歴を追加する

### Chain に Chat history を組み込む  
RunnableWithMessageHistory クラスを使用して Chain の入力に Chat history を組み込む

In [57]:
def get_answer_with_history1(vector_store, q, chat_history, session_id='unused', k=5):
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
    from langchain_core.runnables import RunnablePassthrough, RunnableParallel
    from langchain_openai import ChatOpenAI
    from operator import itemgetter
    from langchain_core.output_parsers import StrOutputParser
    from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
    from langchain_core.chat_history import BaseChatMessageHistory
    from langchain_core.runnables.history import RunnableWithMessageHistory

    # Chat model
    model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    # Retriever
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    # Prompt template
    system_message = """以下の参考用のテキストの一部を参照して、質問に回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。

    {context}"""
    human_message = "質問：{query}"

    chat_prompt = ChatPromptTemplate.from_messages([

        (
            "system", system_message
        ),
        MessagesPlaceholder("chat_history"),
        (
            "human", human_message
        )
    ])

    # ユーザーのクエリと retriever が取得した Documents を出力する Runnable
    add_context = RunnablePassthrough.assign(context=itemgetter("query") | retriever)

    # add_context のテスト出力
    test_output = add_context.invoke({'query': q})
    print(f"TestOutput: {test_output}")
    # テスト出力には含まれないが、以下の処理では RunnableWithMessageHistory によって "chat_history" キーにチャット履歴が渡される

    # Chain を定義
    rag_chain = add_context | chat_prompt | model | StrOutputParser()

    # RunnableWithMessageHistory: Chat history を入力に追加して Chain を実行するインスタンス
    runnable_with_history = RunnableWithMessageHistory(
        rag_chain,
        lambda session_id: chat_history, # session_id を受け取って対応する chat message history インスタンス (BaseChatMessageHistory) を返す関数
        input_messages_key="query",
        history_messages_key="chat_history",
    )

    # Chain の実行
    answer = runnable_with_history.invoke({'query': q}, config={"configurable": {"session_id": session_id}})
    #answer = runnable_with_history.invoke({'query': q}, config={"configurable": {"session_id": session_id}, 'callbacks': [ConsoleCallbackHandler()]})
    return answer

`runnable_with_history` の実行では、以下のような処理が行われる  

1. `RunnableWithMessageHistory` により `{'query': <ユーザーのクエリ>, 'chat_history': <会話履歴の Messages>}` が `add_context` に入力される
2. `add_context` が `<ユーザーのクエリ>` を `retriever` に渡して関連する Documents を取得し、`'query': <ユーザーのクエリ>, 'chat_history': <会話履歴の Messages>, context: <retriever が取得した Documents>}` を出力する
3. `chat_prompt` がそれを入力として受け取り、テンプレートの各変数 `context` `query` `chat_history`に対応するキーの値を代入して、PromptValue を出力する
4. `model` がそれを入力として受け取り、LLM の API を呼び出す



In [58]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

# Chat history
chat_history = ChatMessageHistory()

import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break

    answer = get_answer_with_history1(vector_store, q, chat_history, 'b234')
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')

Write Quit or Exit to quit.
Question #1: 育児休暇取得できますか＿
TestOutput: {'query': '育児休暇取得できますか＿', 'context': [Document(id='e281ba50-4e31-4a66-b8ad-bf77f84dd6bc', metadata={'page': 12.0, 'source': 'files/employment_regulations.pdf'}, page_content='第 3 条 （育児休職の対象） \n1. 育児休職は、従業員が満 1 歳未満の子を養育するために取得することができま\nす。 \n2. 特別な事情がある場合、育児休職は最長で子が満 2 歳になるまで延長すること\nが可能です。 \n第 4 条 （育児休職の申請および取得） \n1. 育児休職を希望する従業員は、原則として休職開始予定日の 1 ヶ月前までに、\n所定の申請書を人事部門に提出しなければなりません。 \n2. 申請書には、育児休職の開始日および終了予定日を明記する必要があります。 \n3. 従業員は、配偶者が同時に育児休職を取得する場合も同様に申請することがで\nきます。'), Document(id='f0a2b2b0-5c03-4f1f-a07a-fb5bc0954da7', metadata={'page': 13.0, 'source': 'files/employment_regulations.pdf'}, page_content='門が個別に案内します。 \n3. 育児休職中に積極的にスキルアップやリモートでの業務参加を希望する従業員\nには、会社がサポートを提供することがあります。 \n第 6 条 （育児休職からの復職） \n1. 従業員が育児休職から復職する際、原則として休職前と同一の職務および勤務\n地に復帰します。 \n2. 復職後の業務負担が過重にならないよう、短時間勤務やテレワークなどの柔軟\nな働き方を希望する場合は、会社が可能な範囲で対応します。 \n第 7 条 （育児休職の延長） \n1. 特別な事情がある場合（例：保育所 の入所待機、配偶者 の介護など） 、育児休\n職の延長を申請することができます。'), Document(id='47f0919

## Retriever にも会話履歴を考慮させる
*  上の方法では、モデルに最終的に渡すプロンプトに会話履歴を含めることはできるが、Retriever にドキュメントを検索させる際には会話履歴を含まない最新の入力だけが使われるため、必ずしも適切なドキュメントが検索されないことがある (前の会話を受けた指示語が入力文に含まれている場合など)
*  履歴を含んだ入力から、いったんモデルを使って検索のための文を生成し、それを使って Retriever に検索させることで適切なドキュメントを検索させることができる。

In [59]:
def get_answer_with_history2(vector_store, q, chat_history, session_id='unused', k=20):
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
    from langchain_core.runnables import RunnablePassthrough, RunnableParallel
    from langchain_openai import ChatOpenAI
    from operator import itemgetter
    from langchain_core.output_parsers import StrOutputParser
    from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
    from langchain_core.chat_history import BaseChatMessageHistory
    from langchain_core.runnables.history import RunnableWithMessageHistory

    from langchain.chains import create_history_aware_retriever

    # Chat model
    model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    # Retriever
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    # 検索用にクエリを書き換えるためのプロンプト
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    # 会話履歴を考慮する Retriever を作成
    history_aware_retriever = create_history_aware_retriever(
        model, retriever, contextualize_q_prompt
    )

    # ユーザーのクエリに回答させるための Prompt template
    system_message = """以下の参考用のテキストの一部を参照して、質問に回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。

    {context}"""
    human_message = "質問：{input}"

    chat_prompt = ChatPromptTemplate.from_messages([

        (
            "system", system_message
        ),
        MessagesPlaceholder("chat_history"),
        (
            "human", human_message
        )
    ])

    # ユーザーのクエリと history_aware_retriever が取得した Documents を出力する Runnable
    add_context = RunnablePassthrough.assign(context=history_aware_retriever)

    # Chain を定義
    rag_chain = add_context | chat_prompt | model | StrOutputParser()

    runnable_with_history = RunnableWithMessageHistory(
        rag_chain,
        lambda session_id: chat_history, # session_id を受け取って対応する chat message history インスタンス (BaseChatMessageHistory) を返す関数
        input_messages_key="input",
        history_messages_key="chat_history",
    )

    # Chain の実行
    answer = runnable_with_history.invoke({'input': q}, config={"configurable": {"session_id": session_id}})
    # answer = runnable_with_history.invoke({'input': q}, config={"configurable": {"session_id": session_id}, 'callbacks': [ConsoleCallbackHandler()]})
    return answer

In [60]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

# Chat history
chat_history = ChatMessageHistory()

import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break

    answer = get_answer_with_history2(vector_store, q, chat_history, 'a234')
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')

Write Quit or Exit to quit.
Question #1: 育児休暇取得できますか？

Answer: 育児休職は、従業員が満1歳未満の子を養育するために取得することができます。また、特別な事情がある場合には、育児休職を最長で子が満2歳になるまで延長することも可能です。したがって、育児休暇を取得することができます。

 -------------------------------------------------- 

Question #2: その間のサポートありますか？

Answer: はい、育児休職中の従業員には、会社がサポートを提供することがあります。具体的には、育児休職中の研修、メンタリング、キャリアカウンセリングなどのキャリアサポートプログラムが用意されており、職場復帰に向けた準備を行うためのリソースや支援が提供されます。

 -------------------------------------------------- 

Question #3: bye

Answer: 分からない。

 -------------------------------------------------- 

Question #4: quit
Quitting ... bye bye!


## 参考：組み込み関数による Chain の作成
次のように、LangChain に組み込みの関数を使って Chain を作成することもできる。  
ただし、ユーザーからの入力や Chain からの出力のキーが既定のものに制約される。  
[langchain.chains.retrieval.create_retrieval_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html)  
[langchain.chains.combine_documents.stuff.create_stuff_documents_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html)

In [ ]:
def get_answer_with_history3(vector_store, q, chat_history, session_id='unused', k=10):
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
    from langchain_core.runnables import RunnablePassthrough, RunnableParallel
    from langchain_openai import ChatOpenAI
    from operator import itemgetter
    from langchain_core.output_parsers import StrOutputParser
    from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
    from langchain_core.chat_history import BaseChatMessageHistory
    from langchain_core.runnables.history import RunnableWithMessageHistory

    from langchain.chains.combine_documents import create_stuff_documents_chain
    from langchain.chains import create_history_aware_retriever, create_retrieval_chain

    # Chat model
    model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    # Retriever
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    # Chat template
    system_message = """以下の参考用のテキストの一部を参照して、質問に回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。

    {context}"""
    human_message = "質問：{input}" # retriever (BaseRetriever のサブクラス) を使用する場合、ユーザーからの入力のキーは "input" にする必要がある

    chat_prompt = ChatPromptTemplate.from_messages([

        (
            "system", system_message
        ),
        MessagesPlaceholder("chat_history"),
        (
            "human", human_message
        )
    ])
    # Q and A 部分の Chain を作る
    question_answer_chain = create_stuff_documents_chain(model, chat_prompt)

    # Retriever を組み込んだ全体の Chain を作る
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    runnable_with_history = RunnableWithMessageHistory(
        rag_chain,
        lambda session_id: chat_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer", # 組み込み関数で作成した rag_chain の出力では "answer" キーの値として回答本文が格納される
    )

    # Chain の実行
    answer = runnable_with_history.invoke({'input': q}, config={"configurable": {"session_id": session_id}})["answer"]
    #answer = runnable_with_history.invoke({'query': q}, config={"configurable": {"session_id": session_id}, 'callbacks': [ConsoleCallbackHandler()]})
    return answer

In [ ]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

# Chat history
chat_history = ChatMessageHistory()

import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break

    answer = get_answer_with_history3(vector_store, q, chat_history, 'a345')
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')